# [ Excel => DataFrame 저장 다루기 ]

In [4]:
import pandas as pd

# 파일 호출
file_single = '../DATA/Single Worksheet.xlsx'
file_multi = '../DATA/Multiple Worksheets.xlsx'

# excel => DataFrame
pd.read_excel(file_single)

,First Name,Last Name,City,Gender
0,Brandon,James,Miami,M
1,Sean,Hawkins,Denver,M
2,Judy,Day,Los Angeles,F
3,Ashley,Ruiz,San Francisco,F
4,Stephanie,Gomez,Portland,F


In [84]:
# [ sheet_name ] : 시트가 많을 때
pd.read_excel(file_multi, sheet_name='Data 1')
pd.read_excel(file_multi, sheet_name=0)
# sheet_name : 시트명 or 인덱스

,First Name,Last Name,City,Gender
0,Brandon,James,Miami,M
1,Sean,Hawkins,Denver,M
2,Judy,Day,Los Angeles,F
3,Ashley,Ruiz,San Francisco,F
4,Stephanie,Gomez,Portland,F


In [86]:
# sheet_name = None : 모든 시트를 추출
pd.read_excel(file_multi, sheet_name=None)

{'Data 1':   First Name Last Name           City Gender
 0    Brandon     James          Miami      M
 1       Sean   Hawkins         Denver      M
 2       Judy       Day    Los Angeles      F
 3     Ashley      Ruiz  San Francisco      F
 4  Stephanie     Gomez       Portland      F,
 'Data 2':   First Name Last Name           City Gender
 0     Parker     Power        Raleigh      F
 1    Preston  Prescott   Philadelphia      F
 2    Ronaldo   Donaldo         Bangor      M
 3      Megan   Stiller  San Francisco      M
 4     Bustin    Jieber         Austin      F,
 'Data 3':   First Name  Last Name     City Gender
 0     Robert     Miller  Seattle      M
 1       Tara     Garcia  Phoenix      F
 2    Raphael  Rodriguez  Orlando      M}

### [ JSON ] : Javascript Object Notation
 * 자바 스크립트의 데이터 형태, 데이터 교환에 사용
 * 딕셔너리 형태 { 키 : 값, 키 : {키:값, 키:값} }
 * 가독성 위해 array마다 나누어 표현
 * true, false : 소문자!

In [6]:
# 1. 데이터 준비
file_json = '../DATA/nobel.json'

In [9]:
# 2. DF로 저장
nobleDF = pd.read_json(file_json)
nobleDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 646 entries, 0 to 645
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   prizes  646 non-null    object
dtypes: object(1)
memory usage: 5.2+ KB


In [12]:
nobleDF.head(3), nobleDF.tail(3)

(                                              prizes
 0  {'year': '2019', 'category': 'chemistry', 'lau...
 1  {'year': '2019', 'category': 'economics', 'lau...
 2  {'year': '2019', 'category': 'literature', 'la...,
                                                 prizes
 643  {'year': '1901', 'category': 'peace', 'laureat...
 644  {'year': '1901', 'category': 'physics', 'laure...
 645  {'year': '1901', 'category': 'medicine', 'laur...)

In [18]:
# 4. 데이터 추출

# 4.1 [ 행 추출 ] => .iloc[]  or .loc[]
nobleDF.iloc[0]

prizes    {'year': '2019', 'category': 'chemistry', 'lau...
Name: 0, dtype: object

In [27]:
oneData = nobleDF.iloc[0]['prizes']   # 위랑 다른 점은?
oneData   # type : dict

{'year': '2019',
 'category': 'chemistry',
 'laureates': [{'id': '976',
   'firstname': 'John',
   'surname': 'Goodenough',
   'motivation': '"for the development of lithium-ion batteries"',
   'share': '3'},
  {'id': '977',
   'firstname': 'M. Stanley',
   'surname': 'Whittingham',
   'motivation': '"for the development of lithium-ion batteries"',
   'share': '3'},
  {'id': '978',
   'firstname': 'Akira',
   'surname': 'Yoshino',
   'motivation': '"for the development of lithium-ion batteries"',
   'share': '3'}]}

In [28]:
oneData.keys(), oneData.values()

(dict_keys(['year', 'category', 'laureates']),
 dict_values(['2019', 'chemistry', [{'id': '976', 'firstname': 'John', 'surname': 'Goodenough', 'motivation': '"for the development of lithium-ion batteries"', 'share': '3'}, {'id': '977', 'firstname': 'M. Stanley', 'surname': 'Whittingham', 'motivation': '"for the development of lithium-ion batteries"', 'share': '3'}, {'id': '978', 'firstname': 'Akira', 'surname': 'Yoshino', 'motivation': '"for the development of lithium-ion batteries"', 'share': '3'}]]))

In [29]:
# [반복문으로 하나씩 열어서 정리 ] : for로 키 별 값을 출력하기; 복잡,,
for k, v in oneData.items():
    print(f'[{k}]\n ==> {v}')
    if k == 'laureates':
        for d in v:
            for k1, v1 in d.items():
                print(k1, v1)

[year]
 ==> 2019
[category]
 ==> chemistry
[laureates]
 ==> [{'id': '976', 'firstname': 'John', 'surname': 'Goodenough', 'motivation': '"for the development of lithium-ion batteries"', 'share': '3'}, {'id': '977', 'firstname': 'M. Stanley', 'surname': 'Whittingham', 'motivation': '"for the development of lithium-ion batteries"', 'share': '3'}, {'id': '978', 'firstname': 'Akira', 'surname': 'Yoshino', 'motivation': '"for the development of lithium-ion batteries"', 'share': '3'}]
id 976
firstname John
surname Goodenough
motivation "for the development of lithium-ion batteries"
share 3
id 977
firstname M. Stanley
surname Whittingham
motivation "for the development of lithium-ion batteries"
share 3
id 978
firstname Akira
surname Yoshino
motivation "for the development of lithium-ion batteries"
share 3


In [33]:
# [ json_normalize (평탄화) ] : 1차원으로 펼쳐 놓는 것; 위 반복문이랑 같은 결과로!
# 평탄화 : 중첩 데이터의 다중 반복문 ==> 단일 리스트 형태 처리 : 평탄화 / 정규화

# [ record_path ] : 중첩된 레코드를 정규화; dict 내 dict를 펼쳐주는 역할

# [ meta ] : 기존 (상위) 열은 record_path 시 생략, 이를 불러와줌

pd.json_normalize(oneData, record_path='laureates', meta=['year', 'category'])

,id,firstname,surname,motivation,share,year,category
0,976,John,Goodenough,"""for the development of lithium-ion batteries""",3,2019,chemistry
1,977,M. Stanley,Whittingham,"""for the development of lithium-ion batteries""",3,2019,chemistry
2,978,Akira,Yoshino,"""for the development of lithium-ion batteries""",3,2019,chemistry


In [34]:
# [정리]
# 한 덩어리로 들어온 'laureates'를 여는 게 record_path
# 과정에서 사라진 열을 불러오는 게 'meta'

In [ ]:
# pd.json_normalize(data=file_json) 
# 엥 왜 안되지 : 딕셔너리를 안 깠음! oneData가 그 과정

### - 모든 행을 정규화/평탄화 하기

In [39]:
# column 읽기  => DF.col or DF[col]
nobleDF.prizes  # type : Series

0      {'year': '2019', 'category': 'chemistry', 'lau...
1      {'year': '2019', 'category': 'economics', 'lau...
2      {'year': '2019', 'category': 'literature', 'la...
3      {'year': '2019', 'category': 'peace', 'laureat...
4      {'year': '2019', 'category': 'physics', 'overa...
                             ...                        
641    {'year': '1901', 'category': 'chemistry', 'lau...
642    {'year': '1901', 'category': 'literature', 'la...
643    {'year': '1901', 'category': 'peace', 'laureat...
644    {'year': '1901', 'category': 'physics', 'laure...
645    {'year': '1901', 'category': 'medicine', 'laur...
Name: prizes, Length: 646, dtype: object

In [42]:
# KeyError: 모든 행에 record_path 에 해당하는 키가 존재해야함!

pd.json_normalize(nobleDF.prizes, record_path='laureates')

KeyError: "Key 'laureates' not found. If specifying a record_path, all elements of data should have the path."

In [47]:
for data in nobleDF.prizes:
    print(type(data), data.keys(), end='\n\n')
    # 해당 dict에 laureates 키가 없다면 추가할 것
    data.setdefault('laureates', [])   # .get 과 유사한 함수, (키, 값)을 입력
print(nobleDF)

<class 'dict'> dict_keys(['year', 'category', 'laureates'])

<class 'dict'> dict_keys(['year', 'category', 'laureates'])

<class 'dict'> dict_keys(['year', 'category', 'laureates'])

<class 'dict'> dict_keys(['year', 'category', 'laureates'])

<class 'dict'> dict_keys(['year', 'category', 'overallMotivation', 'laureates'])

<class 'dict'> dict_keys(['year', 'category', 'laureates'])

<class 'dict'> dict_keys(['year', 'category', 'laureates'])

<class 'dict'> dict_keys(['year', 'category', 'laureates'])

<class 'dict'> dict_keys(['year', 'category', 'laureates'])

<class 'dict'> dict_keys(['year', 'category', 'laureates'])

<class 'dict'> dict_keys(['year', 'category', 'overallMotivation', 'laureates'])

<class 'dict'> dict_keys(['year', 'category', 'laureates'])

<class 'dict'> dict_keys(['year', 'category', 'laureates'])

<class 'dict'> dict_keys(['year', 'category', 'laureates'])

<class 'dict'> dict_keys(['year', 'category', 'laureates'])

<class 'dict'> dict_keys(['year', 'category

In [51]:
# laureates 키를 다 넣었으므로 record_path가 가능해진다
nobleDF2 = pd.json_normalize(nobleDF.prizes, record_path='laureates', meta=["year", "category"])
nobleDF2, nobleDF2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 950 entries, 0 to 949
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          950 non-null    object
 1   firstname   950 non-null    object
 2   surname     921 non-null    object
 3   motivation  950 non-null    object
 4   share       950 non-null    object
 5   year        950 non-null    object
 6   category    950 non-null    object
dtypes: object(7)
memory usage: 52.1+ KB


(      id       firstname      surname  \
 0    976            John   Goodenough   
 1    977      M. Stanley  Whittingham   
 2    978           Akira      Yoshino   
 3    982         Abhijit     Banerjee   
 4    983          Esther        Duflo   
 ..   ...             ...          ...   
 945  569           Sully    Prudhomme   
 946  462           Henry       Dunant   
 947  463        Frédéric        Passy   
 948    1  Wilhelm Conrad      Röntgen   
 949  293            Emil  von Behring   
 
                                             motivation share  year    category  
 0       "for the development of lithium-ion batteries"     3  2019   chemistry  
 1       "for the development of lithium-ion batteries"     3  2019   chemistry  
 2       "for the development of lithium-ion batteries"     3  2019   chemistry  
 3    "for their experimental approach to alleviatin...     3  2019   economics  
 4    "for their experimental approach to alleviatin...     3  2019   economics  
 .

In [52]:
### (7) JSON의 다양한 방식의 DataFrame 저장하기 : orient 파라미터

In [63]:
json_data = '''
[
    {"id":"1", "name":"sravan", "age":22},
    {"id":"2", "name":"harsha", "age":22},
    {"id":"3", "name":"deepika", "age":21},
    {"id":"4", "name":"jyothika", "age":23}
]'''

In [64]:
# orient = 'records' 설정 => JSON 파일 데이터를 행 단위로 읽어서 DataFrame에 저장
# recordS : 파일 형식이 { col : val, col:val, col:val } 를 하나의 행으로 판단
df = pd.read_json(json_data, orient ='records')
print(df)

   id      name  age
0   1    sravan   22
1   2    harsha   22
2   3   deepika   21
3   4  jyothika   23


In [75]:
json_data2 = '''
{
    "student-1" : {"id":"1","name":"sravan","age":22},
    "student-2" : {"id":"2","name":"harsha","age":22},
    "student-3" : {"id":"3","name":"deepika","age":21},
    "student-4" : {"id":"4","name":"jyothika","age":23}
}'''
json_data2

'\n{\n    "student-1" : {"id":"1","name":"sravan","age":22},\n    "student-2" : {"id":"2","name":"harsha","age":22},\n    "student-3" : {"id":"3","name":"deepika","age":21},\n    "student-4" : {"id":"4","name":"jyothika","age":23}\n}'

In [74]:
# 키 : {값} => 키를 인덱스로
df = pd.read_json(json_data2, orient ='index')
print(df)

           id      name  age
student-1   1    sravan   22
student-2   2    harsha   22
student-3   3   deepika   21
student-4   4  jyothika   23


In [77]:
# JSON Format 
json_data3 = '''
[
    [ "1", "sravan",22 ],
    [ "2", "harsha",22 ],
    [ "3", "deepika",21 ],
    [ "4", "jyothika",23 ]
]
'''

In [79]:
# 'values' : 값만 있는 데이터
df = pd.read_json( json_data3, orient='values')
df

,0,1,2
0,1,sravan,22
1,2,harsha,22
2,3,deepika,21
3,4,jyothika,23


In [87]:
# JsonSeriesOrient: TypeAlias = Literal["split", "records", "index", "table"]
# Orient의 종류 :
# https://pandas.pydata.org/docs/reference/api/pandas.read_json.html

# orient의 종류~로 위 내용 보면서 정리하기

In [88]:
# Fin.